In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Cargar archivo original
df = pd.read_csv("estacion1.csv")

# Asegurarse de que la columna 'fecha' esté en formato datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Pivotar el dataset: cada fila será un timestamp y cada columna un sensor
df_pivot = df.pivot_table(index='fecha', columns='sensor_name', values='sensor_dato', aggfunc='first').reset_index()

# Codificar variable categórica: Direccion del Viento (One-Hot)
if 'Direccion del Viento' in df_pivot.columns:
    df_pivot = pd.get_dummies(df_pivot, columns=['Direccion del Viento'], prefix='Viento')

# Convertir todas las columnas posibles a numéricas 
for col in df_pivot.columns:
    if col != 'fecha':
        df_pivot[col] = pd.to_numeric(df_pivot[col], errors='coerce')

# Eliminar filas con CO2 faltante (target)
df_pivot = df_pivot[df_pivot['CO2'].notnull()].copy()

# Guardar columna fecha por separado y escalar el resto
fechas = df_pivot['fecha']
df_num = df_pivot.drop(columns=['fecha'])

# Normalizar entre 0 y 1
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_num), columns=df_num.columns)

# Volver a unir fechas
df_final = pd.concat([fechas.reset_index(drop=True), df_scaled], axis=1)

# Guardar como archivo listo para TFT
df_final.to_csv("data_tft_ready.csv", index=False)

print("✅ Dataset listo para TFT. Tamaño final:", df_final.shape)

✅ Dataset listo para TFT. Tamaño final: (34685, 22)
